[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sunfish256/chatgpt-api-samples/blob/main/demo/10_chatbot_ui.ipynb)

[参考](https://qiita.com/tkmrsksk/items/7362f183138dfb324c50)  
[gradio document](https://gradio.app/creating-a-chatbot/)  
[gradio入門](https://note.com/npaka/n/nb9d4902f8f4d)  

In [ ]:
# ! pip install langchain openai gradio

In [ ]:
import gradio as gr
from langchain.chains import ConversationChain
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate

api_key = 'YOUR_KEY'  # input your openai api key

css = '''
.message.user{
    background: #06c755 !important;
}'''

with gr.Blocks(css=css) as demo:
    chatbot = gr.Chatbot()
    msg = gr.Textbox(label='Message')
    clear = gr.Button('Reset')

    def create_bot():
        llm = ChatOpenAI(model_name='gpt-3.5-turbo', openai_api_key=api_key)
        # Change the templete to customize your bot
        template = '''私と会話してください。
        {history}
        {input}
        '''
        prompt = PromptTemplate(
            input_variables=['history', 'input'],
            template=template
        )
        memory = ConversationBufferMemory(return_messages=True)
        global conversation
        conversation = ConversationChain(llm=llm, prompt=prompt, memory=memory, verbose=True)

    def user(message, history):
        return '', history + [[message, None]]

    def chat(history):
        message = history[-1][0]
        response = conversation.predict(input=message)
        history[-1][1] = response
        return history

    create_bot()
    msg.submit(user, [msg, chatbot], [msg, chatbot], queue=False).then(
        chat, chatbot, chatbot
    )
    clear.click(create_bot, None, chatbot, queue=False)

if __name__ == '__main__':
    demo.launch()